# Datensatz vorbereiten

In [1]:
import pandas
fname = "used_cars_autoscout24.csv"
t = pandas.read_csv(fname)
t["power_ps"] = pandas.to_numeric(t["power_ps"], errors='coerce')
t["price_in_euro"] = pandas.to_numeric(t["price_in_euro"], errors='coerce')
t_bereinigt = t[ ["brand", "mileage_in_km", "power_ps", "price_in_euro"] ]
t_bereinigt = t_bereinigt.dropna(axis="rows")
t_bereinigt = pandas.get_dummies(t_bereinigt)

x = t_bereinigt.drop("price_in_euro", axis="columns")
y = t_bereinigt[ ["price_in_euro"] ]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

from sklearn.preprocessing import MinMaxScaler
s_input = MinMaxScaler(feature_range=(0,1))
s_output = MinMaxScaler(feature_range=(0,1))
s_input.fit(x_train)
s_output.fit(y_train)
x_train_scaled = s_input.transform(x_train)
y_train_scaled = s_output.transform(y_train)
x_test_scaled = s_input.transform(x_test)
y_test_scaled = s_output.transform(y_test)

# Ein NN mit Keras erstellen und trainieren

In [15]:
x_train_scaled.shape[0] / 32

6267.28125

In [2]:
nr_inputs = x_train_scaled.shape[1]

from tensorflow import keras
model = keras.models.Sequential()
model.add( keras.layers.InputLayer(shape=(nr_inputs,)) )
model.add( keras.layers.Dense(1, activation=None) )

model.compile(optimizer="adam", loss="mean_absolute_error")

h = model.fit(x_train_scaled, y_train_scaled, epochs=3)

2024-10-28 19:29:52.799025: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-28 19:29:54.318134: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 1/3


2024-10-28 19:29:55.576280: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-28 19:29:55.576881: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


6268/6268 ━━━━━━━━━━━━━━━━━━━━ 6s 907us/step - loss: 0.0214
Epoch 2/3
6268/6268 ━━━━━━━━━━━━━━━━━━━━ 5s 831us/step - loss: 0.0016
Epoch 3/3
6268/6268 ━━━━━━━━━━━━━━━━━━━━ 5s 866us/step - loss: 0.0016


In [11]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1)              │            50 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50 (200.00 B)

 Trainable params: 50 (200.00 B)

 Non-trainable params: 0 (0.00 B)

In [18]:
type(h)

keras.src.callbacks.history.History

In [19]:
h.history

{'loss': [0.006746484432369471, 0.0015839834231883287, 0.001582715311087668]}

# Modellgüte bestimmen

In [3]:
x_test_scaled.shape

(50139, 49)

In [4]:
x_test_scaled[0]

array([0.01731767, 0.15130261, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

In [6]:
preds_scaled = model.predict( x_test_scaled, verbose=0 )
preds_scaled.shape

(50139, 1)

In [8]:
preds_scaled[:10]

array([[0.00368542],
       [0.00431819],
       [0.00395836],
       [0.00249691],
       [0.00248036],
       [0.00536162],
       [0.00251638],
       [0.00333258],
       [0.00077021],
       [0.00505806]], dtype=float32)

In [10]:
preds_euro = s_output.inverse_transform( preds_scaled )
preds_euro[:10]

array([[21828.502 ],
       [25555.79  ],
       [23436.232 ],
       [14827.728 ],
       [14730.263 ],
       [31702.    ],
       [14942.43  ],
       [19750.139 ],
       [ 4656.8154],
       [29913.918 ]], dtype=float32)

In [11]:
y_test.shape

(50139, 1)

In [36]:
preds_scaled = model.predict( x_test_scaled, verbose=0 )
preds_euro = s_output.inverse_transform( preds_scaled ).flatten()
gt_euro = y_test.values.flatten()

from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
MAE  = mean_absolute_error(gt_euro, preds_euro)
MAPE = mean_absolute_percentage_error(gt_euro, preds_euro)*100
print(f"{MAE=:.2f}€, {MAPE=:.2f}%")

MAE=8847.37€, MAPE=51.78%


In [15]:
preds_euro.shape

(50139,)

In [16]:
gt_euro.shape

(50139,)

In [17]:
preds_euro[:20]

array([21828.502  , 25555.79   , 23436.232  , 14827.728  , 14730.263  ,
       31702.     , 14942.43   , 19750.139  ,  4656.8154 , 29913.918  ,
       22307.668  ,  3660.9456 ,  7951.9272 ,   441.85944, 24621.904  ,
        1976.6394 ,  8512.65   , 12238.87   , 26872.969  , 15716.501  ],
      dtype=float32)

In [18]:
gt_euro[:20]

array([12900., 24550., 21290., 10999., 21450., 30988.,  9799., 16390.,
        9999., 57990., 22890.,  7500., 14950.,  3999., 17990.,  4980.,
        2250.,  8950., 36180., 17990.])

In [21]:
import numpy
MAE = numpy.mean(abs(gt_euro - preds_euro))
MAE

8847.371960436556

In [22]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(gt_euro, preds_euro)

8847.371960436556

In [27]:
(abs(15000 - 10000)/15000)*100

33.33333333333333

In [28]:
(abs(1005000 - 1000000)/1005000)*100

0.4975124378109453

In [32]:
import numpy
MAPE = (numpy.mean(abs(gt_euro - preds_euro)/gt_euro))*100
MAPE

51.77962327542708

In [34]:
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(gt_euro, preds_euro)*100

51.77962327542708

# Codeteile als ein Skript

In [22]:
t["fuel_type"]

0           Petrol
1           Petrol
2           Petrol
3           Petrol
4           Petrol
            ...   
251074      Hybrid
251075      Diesel
251076      Diesel
251077      Hybrid
251078    Electric
Name: fuel_type, Length: 251079, dtype: object

In [17]:
t["color"].nunique()

14

In [7]:
t.dtypes

Unnamed: 0                    int64
brand                        object
model                        object
color                        object
registration_date            object
year                         object
price_in_euro               float64
power_kw                     object
power_ps                    float64
transmission_type            object
fuel_type                    object
fuel_consumption_l_100km     object
fuel_consumption_g_km        object
mileage_in_km               float64
offer_description            object
dtype: object

In [6]:
t.columns

Index(['Unnamed: 0', 'brand', 'model', 'color', 'registration_date', 'year',
       'price_in_euro', 'power_kw', 'power_ps', 'transmission_type',
       'fuel_type', 'fuel_consumption_l_100km', 'fuel_consumption_g_km',
       'mileage_in_km', 'offer_description'],
      dtype='object')

In [14]:
t_bereinigt.columns

Index(['mileage_in_km', 'power_ps', 'price_in_euro', 'brand_alfa-romeo',
       'brand_aston-martin', 'brand_audi', 'brand_bentley', 'brand_bmw',
       'brand_cadillac', 'brand_chevrolet', 'brand_chrysler', 'brand_citroen',
       'brand_dacia', 'brand_daewoo', 'brand_daihatsu', 'brand_dodge',
       'brand_ferrari', 'brand_fiat', 'brand_ford', 'brand_honda',
       'brand_hyundai', 'brand_infiniti', 'brand_isuzu', 'brand_jaguar',
       'brand_jeep', 'brand_kia', 'brand_lada', 'brand_lamborghini',
       'brand_lancia', 'brand_land-rover', 'brand_maserati', 'brand_mazda',
       'brand_mercedes-benz', 'brand_mini', 'brand_mitsubishi', 'brand_nissan',
       'brand_opel', 'brand_peugeot', 'brand_porsche', 'brand_proton',
       'brand_renault', 'brand_rover', 'brand_saab', 'brand_seat',
       'brand_skoda', 'brand_smart', 'brand_ssangyong', 'brand_toyota',
       'brand_volkswagen', 'brand_volvo'],
      dtype='object')

In [30]:
# 1. Beispieldaten für das Modell vorbereiten
import pandas
fname = "used_cars_autoscout24.csv"
t = pandas.read_csv(fname)
t["power_ps"]      = pandas.to_numeric(t["power_ps"], errors='coerce')
t["year"]          = pandas.to_numeric(t["year"], errors='coerce')
t["price_in_euro"] = pandas.to_numeric(t["price_in_euro"], errors='coerce')
t_bereinigt = t[ ["brand", "model", "color", "transmission_type",
                  "mileage_in_km", "power_ps", "year",
                  "price_in_euro"] ]
t_bereinigt = t_bereinigt.dropna(axis="rows")
t_bereinigt = pandas.get_dummies(t_bereinigt)

x = t_bereinigt.drop("price_in_euro", axis="columns")
y = t_bereinigt[ ["price_in_euro"] ]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

from sklearn.preprocessing import MinMaxScaler
s_input = MinMaxScaler(feature_range=(0,1))
s_output = MinMaxScaler(feature_range=(0,1))
s_input.fit(x_train)
s_output.fit(y_train)
x_train_scaled = s_input.transform(x_train)
y_train_scaled = s_output.transform(y_train)
x_test_scaled = s_input.transform(x_test)
y_test_scaled = s_output.transform(y_test)

# 2. NN mit Keras bauen und trainieren
if True:
    nr_inputs = x_train_scaled.shape[1]
    from tensorflow import keras
    model = keras.models.Sequential()
    model.add( keras.layers.InputLayer(shape=(nr_inputs,)) )
    model.add( keras.layers.Dense(10, activation="relu") )
    model.add( keras.layers.Dense(1, activation=None) )
    model.compile(optimizer="adam", loss="mean_absolute_error")
    h = model.fit(x_train_scaled, y_train_scaled, epochs=20)
    
    # 3. Modellgüte bestimmen
    preds_scaled = model.predict( x_test_scaled, verbose=0 )
    preds_euro = s_output.inverse_transform( preds_scaled ).flatten()
    gt_euro = y_test.values.flatten()
    from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
    MAE  = mean_absolute_error(gt_euro, preds_euro)
    MAPE = mean_absolute_percentage_error(gt_euro, preds_euro)*100
    print(f"{MAE=:.2f}€, {MAPE=:.2f}%")

Epoch 1/20
6264/6264 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 0.0017
Epoch 2/20
6264/6264 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 7.9693e-04
Epoch 3/20
6264/6264 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - loss: 7.6040e-04
Epoch 4/20
6264/6264 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 7.4291e-04
Epoch 5/20
6264/6264 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - loss: 6.9695e-04
Epoch 6/20
6264/6264 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 7.0449e-04
Epoch 7/20
6264/6264 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 6.9670e-04
Epoch 8/20
6264/6264 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 6.9360e-04
Epoch 9/20
6264/6264 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 6.9311e-04
Epoch 10/20
6264/6264 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 6.9003e-04
Epoch 11/20
6264/6264 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 6.9057e-04
Epoch 12/20
6264/6264 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 6.9872e-04
Epoch 13/20
6264/6264 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 6.9006e-04
Epoch 14/20
6264/6264 ━━━━━━━━━━━━━━━━━

In [28]:
t_bereinigt.columns

Index(['mileage_in_km', 'power_ps', 'year', 'price_in_euro',
       'brand_alfa-romeo', 'brand_aston-martin', 'brand_audi', 'brand_bentley',
       'brand_bmw', 'brand_cadillac',
       ...
       'color_orange', 'color_red', 'color_silver', 'color_violet',
       'color_white', 'color_yellow', 'transmission_type_Automatic',
       'transmission_type_Manual', 'transmission_type_Semi-automatic',
       'transmission_type_Unknown'],
      dtype='object', length=1380)